# Changing an environment registered in Sinergym

As discussed above, Sinergym has a list of available environments that we can call with `gym.make(<environment_id>)` as long as the sinergym package is imported into the python script.

In [4]:
import gym
import numpy as np

import sinergym
env = gym.make('Eplus-5Zone-hot-continuous-stochastic-v1')

[2022-05-23 16:41:29,038] EPLUS_ENV_5Zone-hot-continuous-stochastic-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2022-05-23 16:41:29,038] EPLUS_ENV_5Zone-hot-continuous-stochastic-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2022-05-23 16:41:29,041] EPLUS_ENV_5Zone-hot-continuous-stochastic-v1_MainThread_ROOT INFO:Updating idf OutPut:Variable and variables XML tree model for BVCTB connection.
[2022-05-23 16:41:29,041] EPLUS_ENV_5Zone-hot-continuous-stochastic-v1_MainThread_ROOT INFO:Updating idf OutPut:Variable and variables XML tree model for BVCTB connection.
[2022-05-23 16:41:29,043] EPLUS_ENV_5Zone-hot-continuous-stochastic-v1_MainThread_ROOT INFO:Setting up extra configuration in building model if exists...
[2022-05-23 16:41:29,043] EPLUS_ENV_5Zone-hot-continuous-stochastic-v1_MainThread_ROOT INFO:Setting up extra configuration in building model if exists

These environment IDs have a number of components defined, not only the building design (IDF), but also the reward function, the action and observation spaces, the defining variables, etc.

If you want a new environment, you can define it from scratch in our [environment list](https://github.com/ugr-sail/sinergym/blob/main/sinergym/__init__.py) and run it locally.

Another option (recommended) is to start from one of our environments and change the components you want. You can combine components you want to change obviously.

The way to do that is to add in the `gym.make(<environment_id>)` those parameters of the constructor of the Sinergym environment we want to change. Let's see what things we can change starting from any environment:

## Adding a new reward

As mentioned above, simply add the appropriate parameters to `gym.make()` after specifying the environment ID.

In [5]:
from sinergym.utils.rewards import LinearReward, ExpReward

env = gym.make('Eplus-5Zone-hot-continuous-v1', reward=ExpReward, reward_kwargs={
                                                                    'temperature_variable': 'Zone Air Temperature (SPACE1-1)',
                                                                    'energy_variable': 'Facility Total HVAC Electricity Demand Rate (Whole Building)',
                                                                    'range_comfort_winter': (20.0, 23.5),
                                                                    'range_comfort_summer': (23.0, 26.0),
                                                                    'energy_weight': 0.1})

[2022-05-23 16:41:30,160] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2022-05-23 16:41:30,160] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2022-05-23 16:41:30,163] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Updating idf OutPut:Variable and variables XML tree model for BVCTB connection.
[2022-05-23 16:41:30,163] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Updating idf OutPut:Variable and variables XML tree model for BVCTB connection.
[2022-05-23 16:41:30,165] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Setting up extra configuration in building model if exists...
[2022-05-23 16:41:30,165] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Setting up extra configuration in building model if exists...
/usr/local/lib/python3.9/dist-packages/gym/logger.py:34: UserW

You have to specify the **reward** class you are going to use in the environment. A reward function class has several parameters that can be specified by user like temperature variables, weights, etc. Depending on the reward function you are using. In order to be able to define it, we have **reward_kwargs** parameter. See [reward documentation](https://ugr-sail.github.io/sinergym/compilation/html/pages/rewards.html) for more information about reward classes and how to create a new one. 

## Adding other new components to the environment

In the same way that we can change the default reward function, as we have done in the second example, 
it is possible to substitute other default values of the environment ID. 

You can change the weather file, the number of timesteps an action repeats (default 1), 
the last n episodes you want to be stored in the Sinergym output folder (default 10), 
the name of the environment or the variability in stochastic environments:

In [6]:
env = gym.make('Eplus-datacenter-cool-continuous-stochastic-v1', 
                weather_file='ESP_Granada.084190_SWEC.epw',
                weather_variability=(1.0,0.0,0.001),
                env_name='new_env_name',
                act_repeat=4,
                max_ep_data_store_num = 20)

[2022-05-23 16:41:31,228] EPLUS_ENV_new_env_name_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2022-05-23 16:41:31,231] EPLUS_ENV_new_env_name_MainThread_ROOT INFO:Updating idf OutPut:Variable and variables XML tree model for BVCTB connection.
[2022-05-23 16:41:31,234] EPLUS_ENV_new_env_name_MainThread_ROOT INFO:Setting up extra configuration in building model if exists...
/usr/local/lib/python3.9/dist-packages/gym/logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


## Adding more configuration definition

You can even add a dictionary with extra parameters that modify the IDF you use before it is used in the simulations (or overwrite an existing one).

This new IDF version, which also adapts to the new weather you put in, is saved in the Sinergym output folder, leaving the original intact:

In [7]:
extra_conf={
    'timesteps_per_hour':6,
    'runperiod':(1,1,1991,2,1,1992),
    'action_definition': {
        'ThermostatSetpoint:DualSetpoint': [{
            'name': 'West-DualSetP-RL',
            'heating_name': 'West-HtgSetP-RL',
            'cooling_name': 'West-ClgSetP-RL',
            'zones': ['West Zone']
        },
            {
            'name': 'East-DualSetP-RL',
            'heating_name': 'East-HtgSetP-RL',
            'cooling_name': 'East-ClgSetP-RL',
            'zones': ['East Zone']
        }]
    }
}

env = gym.make('Eplus-datacenter-cool-continuous-stochastic-v1', 
                config_params=extra_conf
                )

[2022-05-23 16:41:32,273] EPLUS_ENV_datacenter-cool-continuous-stochastic-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2022-05-23 16:41:32,276] EPLUS_ENV_datacenter-cool-continuous-stochastic-v1_MainThread_ROOT INFO:Updating idf OutPut:Variable and variables XML tree model for BVCTB connection.
[2022-05-23 16:41:32,278] EPLUS_ENV_datacenter-cool-continuous-stochastic-v1_MainThread_ROOT INFO:Setting up extra configuration in building model if exists...
/usr/local/lib/python3.9/dist-packages/gym/logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


For more information about extra configuration parameters, see our [Extra configuration documentation](https://ugr-sail.github.io/sinergym/compilation/html/pages/extra-configuration.html)